In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

In [31]:
###-----------------
### Import Libraries
###-----------------

import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import tensorflow as tf

from sklearn.metrics import accuracy_score

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [41]:
###----------------------
### Some basic parameters
###----------------------


inpDir = '../input'
outDir = '../output'
modelDir = 'models'
subDir = 'flower_photos'
altName = 'torch_all_in_1'
valDir = 'valid_flowers'

RANDOM_STATE = 24 # for initialization ----- REMEMBER: to remove at the time of promotion to production
tf.random.set_seed(RANDOM_STATE) # setting for Tensorflow as well

TEST_SIZE = 0.2

ALPHA = 0.001
EPOCHS = 10 # number of cycles to run
FLIP_MODE = 'horizontal_and_vertical'
ROTATION = (-0.1, 0.1)
BATCH_SIZE = 8 # inline of Training Rows being 60000
IMG_HEIGHT = 200 
IMG_WIDTH = 200 

# Set parameters for decoration of plots
params = {'legend.fontsize' : 'medium',
          'figure.figsize'  : (15,10),
          'axes.labelsize'  : 'medium',
          'axes.titlesize'  :'large',
          'xtick.labelsize' :'medium',
          'ytick.labelsize' :'medium',
         }

CMAP = plt.cm.coolwarm

plt.rcParams.update(params) # update rcParams

plt.style.use('seaborn-v0_8-darkgrid') # plt.style.use('ggplot')

## Basic Hygiene

In [5]:
physical_devices = tf.config.list_physical_devices('GPU')

if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [6]:
print (physical_devices)

[]


In [7]:
# #Delete these lines

# labels = []
# for count, data in enumerate(train_ds):

#     imgs, lbls = data
#     #print(count, ':', imgs.shape, lbls.shape)\
#     labels.append(lbls)

# unique, A, counts = tf.unique_with_counts(tf.concat(labels, axis=0).numpy()) # get counts
# unique, A, counts

In [8]:
def fn_plot_label(tr_ds, ts_ds):

    plt.figure(figsize = (15,5)) # instantiate the figure

    plt.subplot(1,2,1) # first out of 2

    train_labels = tf.concat([lbl for img, lbl in tr_ds], axis = 0).numpy() # get the labels

    unique, _, counts = tf.unique_with_counts(train_labels) # get counts

    plt.bar(range(len(unique)), counts, align='center', color = 'DarkBlue') # barplot the counts

    plt.xticks(range(len(unique)), class_names)

    plt.title('Training Set')

    plt.subplot(1,2,2)

    test_labels = tf.concat([lbl for img, lbl in ts_ds], axis = 0).numpy()

    unique, _, counts = tf.unique_with_counts(test_labels)

    plt.bar(range(len(unique)), counts, align='center', color = 'Orange')

    plt.xticks(range(len(unique)), class_names)

    plt.title('Test Set')


In [9]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

data_dir = tf.keras.utils.get_file(origin=dataset_url,
                                   fname='flower_photos',
                                   untar=True)
data_dir = pathlib.Path(data_dir)


# data_dir = os.path.join(inpDir, subDir)
data_dir

WindowsPath('C:/Users/Administrator.DAI-PC2/.keras/datasets/flower_photos')

In [10]:
os.listdir(data_dir)

['daisy', 'dandelion', 'LICENSE.txt', 'roses', 'sunflowers', 'tulips']

In [11]:
# create training data
train_ds =tf.keras.preprocessing.image_dataset_from_directory(
    data_dir, # path the the data directory
    validation_split=TEST_SIZE, # what ratio of validation data
    subset='training', # purpose
    seed=RANDOM_STATE,
    image_size=[IMG_HEIGHT, IMG_WIDTH], ## @@@ WHAT!
    batch_size=BATCH_SIZE
)
# test data
test_ds =tf.keras.preprocessing.image_dataset_from_directory(
    data_dir, # path the the data directory
    validation_split=TEST_SIZE, # what ratio of validation data
    subset='validation', # purpose
    seed=RANDOM_STATE,
    image_size=[IMG_HEIGHT, IMG_WIDTH], ## @@@ WHAT!
    batch_size=BATCH_SIZE
)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [12]:
# is it picking class names
class_names = train_ds.class_names
class_names

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

In [13]:
class_dict = {k: v for k, v in enumerate(class_names)}
class_dict

{0: 'daisy', 1: 'dandelion', 2: 'roses', 3: 'sunflowers', 4: 'tulips'}

In [14]:
num_classes = len(class_names)
num_classes

5

In [15]:
# plt.figure(figsize=(15,8))

# for images, labels in train_ds.take(1):
#     for i in range (BATCH_SIZE):
#         plt.subplot(int(BATCH_SIZE/8), 8, i +1)
#         plt.grid(False)
#         plt.imshow(images[i].numpy().astype('uint8'))
#         plt.title(class_names[labels[i]])
#         plt.axis('off')
#     plt.tight_layout()
# plt.show()

In [16]:
# plt.figure(figsize=(15,8))

# for images, labels in test_ds.take(1): # get me one batch

#     for i in range (BATCH_SIZE): # loop over batch

#         plt.subplot(int(BATCH_SIZE/8), 8, i +1) # access the axis

#         plt.grid(False) # no to grid

#         plt.imshow(images[i].numpy().astype('uint8')) # show image convert to numpy and int

#         plt.title(class_names[labels[i]])

#         plt.axis('off')

#     plt.tight_layout()

# plt.show()

In [17]:
# fn_plot_label(train_ds, test_ds)

In [18]:
input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)
input_shape

(200, 200, 3)

In [19]:
tf.keras.applications.DenseNet169?

Signature:
tf.keras.applications.DenseNet169(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax',
)
Docstring:
Instantiates the Densenet169 architecture.

Reference:
- [Densely Connected Convolutional Networks](
    https://arxiv.org/abs/1608.06993) (CVPR 2017)

Optionally loads weights pre-trained on ImageNet.
Note that the data format convention used by the model is
the one specified in your Keras config at `~/.keras/keras.json`.

Note: each Keras Application expects a specific kind of input preprocessing.
For DenseNet, call `tf.keras.applications.densenet.preprocess_input` on your
inputs before passing them to the model.

Args:
  include_top: whether to include the fully-connected
    layer at the top of the network.
  weights: one of `None` (random initialization),
    'imagenet' (pre-training on ImageNet),
    or the path to the weights file to be loaded.
  input_tensor:

In [20]:
densenet_model = tf.keras.applications.DenseNet169(weights='imagenet',
                                                include_top=False,
                                                input_shape=input_shape)

In [21]:
#for layer in densenet_model.layers:
   # layer.trainable=False

In [22]:
densenet_model.summary()

Model: "densenet169"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 200, 200, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 206, 206, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 100, 100, 64)         9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 100, 100, 64)         256       ['conv1/conv[0][0]']

                                                                                                  
 conv4_block4_concat (Conca  (None, 12, 12, 384)          0         ['conv4_block3_concat[0][0]', 
 tenate)                                                             'conv4_block4_2_conv[0][0]'] 
                                                                                                  
 conv4_block5_0_bn (BatchNo  (None, 12, 12, 384)          1536      ['conv4_block4_concat[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_block5_0_relu (Activ  (None, 12, 12, 384)          0         ['conv4_block5_0_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv4_blo

 D)                                                                                               
                                                                                                  
 conv4_block8_concat (Conca  (None, 12, 12, 512)          0         ['conv4_block7_concat[0][0]', 
 tenate)                                                             'conv4_block8_2_conv[0][0]'] 
                                                                                                  
 conv4_block9_0_bn (BatchNo  (None, 12, 12, 512)          2048      ['conv4_block8_concat[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_block9_0_relu (Activ  (None, 12, 12, 512)          0         ['conv4_block9_0_bn[0][0]']   
 ation)                                                                                           
          

 conv4_block12_2_conv (Conv  (None, 12, 12, 32)           36864     ['conv4_block12_1_relu[0][0]']
 2D)                                                                                              
                                                                                                  
 conv4_block12_concat (Conc  (None, 12, 12, 640)          0         ['conv4_block11_concat[0][0]',
 atenate)                                                            'conv4_block12_2_conv[0][0]']
                                                                                                  
 conv4_block13_0_bn (BatchN  (None, 12, 12, 640)          2560      ['conv4_block12_concat[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv4_block13_0_relu (Acti  (None, 12, 12, 640)          0         ['conv4_block13_0_bn[0][0]']  
 vation)  

                                                                                                  
 conv4_block16_2_conv (Conv  (None, 12, 12, 32)           36864     ['conv4_block16_1_relu[0][0]']
 2D)                                                                                              
                                                                                                  
 conv4_block16_concat (Conc  (None, 12, 12, 768)          0         ['conv4_block15_concat[0][0]',
 atenate)                                                            'conv4_block16_2_conv[0][0]']
                                                                                                  
 conv4_block17_0_bn (BatchN  (None, 12, 12, 768)          3072      ['conv4_block16_concat[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv4_blo

...

                                                                                                  
 conv5_block15_concat (Conc  (None, 6, 6, 1120)           0         ['conv5_block14_concat[0][0]',
 atenate)                                                            'conv5_block15_2_conv[0][0]']
                                                                                                  
 conv5_block16_0_bn (BatchN  (None, 6, 6, 1120)           4480      ['conv5_block15_concat[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv5_block16_0_relu (Acti  (None, 6, 6, 1120)           0         ['conv5_block16_0_bn[0][0]']  
 vation)                                                                                          
                                                                                                  
 conv5_blo

 2D)                                                                                              
                                                                                                  
 conv5_block19_concat (Conc  (None, 6, 6, 1248)           0         ['conv5_block18_concat[0][0]',
 atenate)                                                            'conv5_block19_2_conv[0][0]']
                                                                                                  
 conv5_block20_0_bn (BatchN  (None, 6, 6, 1248)           4992      ['conv5_block19_concat[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv5_block20_0_relu (Acti  (None, 6, 6, 1248)           0         ['conv5_block20_0_bn[0][0]']  
 vation)                                                                                          
          

 conv5_block23_2_conv (Conv  (None, 6, 6, 32)             36864     ['conv5_block23_1_relu[0][0]']
 2D)                                                                                              
                                                                                                  
 conv5_block23_concat (Conc  (None, 6, 6, 1376)           0         ['conv5_block22_concat[0][0]',
 atenate)                                                            'conv5_block23_2_conv[0][0]']
                                                                                                  
 conv5_block24_0_bn (BatchN  (None, 6, 6, 1376)           5504      ['conv5_block23_concat[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv5_block24_0_relu (Acti  (None, 6, 6, 1376)           0         ['conv5_block24_0_bn[0][0]']  
 vation)  

                                                                                                  
 conv5_block27_2_conv (Conv  (None, 6, 6, 32)             36864     ['conv5_block27_1_relu[0][0]']
 2D)                                                                                              
                                                                                                  
 conv5_block27_concat (Conc  (None, 6, 6, 1504)           0         ['conv5_block26_concat[0][0]',
 atenate)                                                            'conv5_block27_2_conv[0][0]']
                                                                                                  
 conv5_block28_0_bn (BatchN  (None, 6, 6, 1504)           6016      ['conv5_block27_concat[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv5_blo

 vation)                                                                                          
                                                                                                  
 conv5_block31_2_conv (Conv  (None, 6, 6, 32)             36864     ['conv5_block31_1_relu[0][0]']
 2D)                                                                                              
                                                                                                  
 conv5_block31_concat (Conc  (None, 6, 6, 1632)           0         ['conv5_block30_concat[0][0]',
 atenate)                                                            'conv5_block31_2_conv[0][0]']
                                                                                                  
 conv5_block32_0_bn (BatchN  (None, 6, 6, 1632)           6528      ['conv5_block31_concat[0][0]']
 ormalization)                                                                                    
          

In [23]:
def build_model(input_shape, num_classes):

    knl_init = tf.keras.initializers.GlorotUniform(seed = RANDOM_STATE)
    reg_lambda = 0.005


    model= tf.keras.Sequential()

    #Preprocessing

    model.add(tf.keras.layers.Rescaling(1./255.))
    
    model.add(densenet_model)

    # Head

    model.add(tf.keras.layers.Flatten())

    #Dense 1
    model.add(tf.keras.layers.Dense(1664 ,kernel_initializer = knl_init))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation(activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))


    #Dense 2
    model.add(tf.keras.layers.Dense(46 ,activation='relu',kernel_initializer = knl_init))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation(activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))

    #Dense 3
    model.add(tf.keras.layers.Dense(num_classes))

    return model



In [24]:
model=build_model(input_shape,num_classes)
model

In [25]:
model.compile(optimizer=tf.keras.optimizers.Adam(
              learning_rate=ALPHA),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [26]:
history = model.fit(train_ds,
                    validation_data=test_ds,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS)

Epoch 1/10


367/367 [==============================] - 664s 2s/step - loss: 1.6736 - accuracy: 0.3672 - val_loss: 1.2490 - val_accuracy: 0.5232
Epoch 2/10
367/367 [==============================] - 570s 2s/step - loss: 1.4292 - accuracy: 0.4397 - val_loss: 1.4600 - val_accuracy: 0.5300
Epoch 3/10
367/367 [==============================] - 569s 2s/step - loss: 1.1486 - accuracy: 0.5399 - val_loss: 3.2405 - val_accuracy: 0.5082
Epoch 4/10
367/367 [==============================] - 570s 2s/step - loss: 1.0527 - accuracy: 0.5947 - val_loss: 0.9541 - val_accuracy: 0.6104
Epoch 5/10
367/367 [==============================] - 572s 2s/step - loss: 1.0036 - accuracy: 0.6148 - val_loss: 1.2899 - val_accuracy: 0.5531
Epoch 6/10
367/367 [==============================] - 578s 2s/step - loss: 0.9564 - accuracy: 0.6625 - val_loss: 1.0537 - val_accuracy: 0.6431
Epoch 7/10
367/367 [==============================] - 573s 2s/step - loss: 0.8926 - accuracy: 0.6604 - val_loss: 1.1860 - val_accuracy: 0.66

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 200, 200, 3)       0         
                                                                 
 densenet169 (Functional)    (None, 6, 6, 1664)        12642880  
                                                                 
 flatten (Flatten)           (None, 59904)             0         
                                                                 
 dense (Dense)               (None, 1664)              99681920  
                                                                 
 batch_normalization (Batch  (None, 1664)              6656      
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 1664)              0         
                                                        

In [29]:
#tf.keras.utils.plot_model(model, 'model.png', show_shapes=True,
 #                         show_dtype=True, dpi=96, show_layers_activations=False)

In [33]:
yhat = model.predict(test_ds)

ytest = np.concatenate([y for X, y in test_ds], axis = 0).squeeze()

y_pred = yhat.argmax(axis = 1)

print('Accuracy score on test data : {:.4f}'.format(accuracy_score(ytest, y_pred)))

92/92 [==============================] - 29s 312ms/step
Accuracy score on test data : 0.2030


## Validation Data

In [42]:
#VAlidation Data

val_dir = os.path.join(inpDir, valDir)

val_ds = tf.keras.utils.image_dataset_from_directory(val_dir,
                                                    validation_split=None,
                                                    seed=RANDOM_STATE,
                                                    image_size=(IMG_HEIGHT,IMG_WIDTH),
                                                    batch_size=BATCH_SIZE)

Found 231 files belonging to 5 classes.


In [43]:
test_loss, test_acc = model.evaluate(val_ds, verbose =1 )

29/29 [==============================] - 14s 351ms/step - loss: 1.1106 - accuracy: 0.6580


In [44]:
yhat = model.predict(val_ds)
y_test = np.concatenate([y for X, y in val_ds],axis=0).squeeze()

y_pred = yhat.argmax(axis=1)
print('Accuracy Score on Test Data : {:.4f}'.format(accuracy_score(y_test,y_pred)))

29/29 [==============================] - 11s 301ms/step
Accuracy Score on Test Data : 0.1948
